# Lab 08: Function Calling and Tool Integration

**Course:** Generative AI for Banking Sector  
**Institution:** Banco Nacional de Costa Rica (BNCR)  
**Instructor:** Manuela Larrea  
**Duration:** 3 hours

---

## Learning Objectives

By the end of this lab, you will be able to:

1. Understand Azure OpenAI function calling capabilities
2. Define function schemas for banking operations
3. Implement function execution handlers
4. Build a banking assistant that can perform actions
5. Handle multi-step function calls
6. Implement error handling and validation
7. Create secure function calling patterns

---

## Azure Infrastructure

```
╔══════════════════════════════════════════════════════════════════════════╗
║                    LAB 08 - AZURE INFRASTRUCTURE                         ║
╚══════════════════════════════════════════════════════════════════════════╝

┌─────────────────────────────────────────────────────────────────────────┐
│                        YOU (Jupyter Notebook)                            │
│                                                                           │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │  Python Functions (Tools)                                        │   │
│  │  • get_account_balance()                                         │   │
│  │  • get_transaction_history()                                     │   │
│  │  • transfer_funds()                                              │   │
│  └──────────────────────────────────────────────────────────────────┘   │
└────────────────────────────────┬────────────────────────────────────────┘
                                 │
                                 │ Function Definitions + User Query
                                 ▼
                    ┌────────────────────────────┐
                    │   Azure OpenAI Service     │
                    │                            │
                    │  ┌──────────────────────┐  │
                    │  │  GPT-4               │  │
                    │  │  (Function Calling)  │  │
                    │  └──────────────────────┘  │
                    └────────────────────────────┘
                                 │
                                 │ Function Call Instructions
                                 ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                     Local Function Execution                             │
│                     (Simulated Banking API)                              │
└─────────────────────────────────────────────────────────────────────────┘

📊 Resources Used:
  • Azure OpenAI Service (GPT-4)
  • Local Python functions (no external APIs)

💰 Estimated Cost: ~$2.00 per lab session
```

In [ ]:
import os
import sys
import json
from typing import Dict, List, Optional
from datetime import datetime, timedelta
import random
from openai import AzureOpenAI
from dotenv import load_dotenv

sys.path.append('../../utils')
from azure_openai_helper import AzureOpenAIClient

load_dotenv()
client = AzureOpenAIClient()

print("✓ Setup complete")

## Part 1: Understanding Function Calling

Function calling allows GPT models to:
1. Understand when to call a function
2. Extract parameters from natural language
3. Format function calls with correct arguments
4. Return structured data

The model doesn't actually execute functions - it tells you what to call and with what parameters.

## Part 2: Simulated Banking API

Let's create mock banking functions that simulate real banking operations.

In [ ]:
# Simulated database
MOCK_ACCOUNTS = {
    "ACC12345": {
        "account_number": "ACC12345",
        "account_type": "Checking",
        "balance": 5420.50,
        "currency": "USD",
        "status": "active"
    },
    "ACC67890": {
        "account_number": "ACC67890",
        "account_type": "Savings",
        "balance": 15800.00,
        "currency": "USD",
        "status": "active"
    }
}

MOCK_TRANSACTIONS = {
    "ACC12345": [
        {"date": "2024-01-15", "description": "Salary Deposit", "amount": 3500.00, "type": "credit"},
        {"date": "2024-01-16", "description": "Grocery Store", "amount": -120.50, "type": "debit"},
        {"date": "2024-01-17", "description": "Electric Bill", "amount": -85.00, "type": "debit"},
        {"date": "2024-01-18", "description": "Restaurant", "amount": -45.00, "type": "debit"},
    ],
    "ACC67890": [
        {"date": "2024-01-10", "description": "Transfer from Checking", "amount": 1000.00, "type": "credit"},
        {"date": "2024-01-15", "description": "Interest Payment", "amount": 12.50, "type": "credit"},
    ]
}

# Banking functions
def get_account_balance(account_number: str) -> Dict:
    """
    Get the current balance of a bank account
    
    Args:
        account_number: The account number to query
    
    Returns:
        Dictionary with account balance information
    """
    if account_number not in MOCK_ACCOUNTS:
        return {"error": "Account not found"}
    
    account = MOCK_ACCOUNTS[account_number]
    return {
        "account_number": account["account_number"],
        "account_type": account["account_type"],
        "balance": account["balance"],
        "currency": account["currency"],
        "status": account["status"]
    }

def get_transaction_history(account_number: str, limit: int = 10) -> Dict:
    """
    Get recent transaction history for an account
    
    Args:
        account_number: The account number to query
        limit: Maximum number of transactions to return
    
    Returns:
        Dictionary with transaction history
    """
    if account_number not in MOCK_ACCOUNTS:
        return {"error": "Account not found"}
    
    transactions = MOCK_TRANSACTIONS.get(account_number, [])
    return {
        "account_number": account_number,
        "transactions": transactions[:limit],
        "count": len(transactions[:limit])
    }

def transfer_funds(from_account: str, to_account: str, amount: float, description: str = "") -> Dict:
    """
    Transfer funds between accounts
    
    Args:
        from_account: Source account number
        to_account: Destination account number
        amount: Amount to transfer
        description: Optional transfer description
    
    Returns:
        Dictionary with transfer confirmation
    """
    # Validate accounts
    if from_account not in MOCK_ACCOUNTS:
        return {"error": "Source account not found"}
    if to_account not in MOCK_ACCOUNTS:
        return {"error": "Destination account not found"}
    
    # Check balance
    if MOCK_ACCOUNTS[from_account]["balance"] < amount:
        return {"error": "Insufficient funds"}
    
    # Validate amount
    if amount <= 0:
        return {"error": "Invalid amount"}
    
    # Simulate transfer (in production, this would be a database transaction)
    MOCK_ACCOUNTS[from_account]["balance"] -= amount
    MOCK_ACCOUNTS[to_account]["balance"] += amount
    
    transaction_id = f"TXN{random.randint(10000, 99999)}"
    
    return {
        "success": True,
        "transaction_id": transaction_id,
        "from_account": from_account,
        "to_account": to_account,
        "amount": amount,
        "description": description,
        "timestamp": datetime.now().isoformat(),
        "new_balance": MOCK_ACCOUNTS[from_account]["balance"]
    }

def get_account_statement(account_number: str, start_date: str, end_date: str) -> Dict:
    """
    Get account statement for a date range
    
    Args:
        account_number: The account number
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
    
    Returns:
        Dictionary with statement information
    """
    if account_number not in MOCK_ACCOUNTS:
        return {"error": "Account not found"}
    
    transactions = MOCK_TRANSACTIONS.get(account_number, [])
    
    # Filter by date range (simplified)
    filtered = [t for t in transactions if start_date <= t["date"] <= end_date]
    
    total_credits = sum(t["amount"] for t in filtered if t["type"] == "credit")
    total_debits = sum(abs(t["amount"]) for t in filtered if t["type"] == "debit")
    
    return {
        "account_number": account_number,
        "period": {"start": start_date, "end": end_date},
        "transactions": filtered,
        "summary": {
            "total_credits": total_credits,
            "total_debits": total_debits,
            "net_change": total_credits - total_debits
        }
    }

print("✓ Banking functions defined")

# Test the functions
print("\nTest: Get balance")
print(json.dumps(get_account_balance("ACC12345"), indent=2))

## Part 3: Defining Function Schemas

We need to describe our functions to GPT in a specific format (JSON Schema).

In [ ]:
# Define function schemas for OpenAI
FUNCTION_SCHEMAS = [
    {
        "name": "get_account_balance",
        "description": "Get the current balance and details of a bank account",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {
                    "type": "string",
                    "description": "The account number (e.g., ACC12345)"
                }
            },
            "required": ["account_number"]
        }
    },
    {
        "name": "get_transaction_history",
        "description": "Get recent transaction history for an account",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {
                    "type": "string",
                    "description": "The account number"
                },
                "limit": {
                    "type": "integer",
                    "description": "Maximum number of transactions to return (default: 10)",
                    "default": 10
                }
            },
            "required": ["account_number"]
        }
    },
    {
        "name": "transfer_funds",
        "description": "Transfer money between two accounts",
        "parameters": {
            "type": "object",
            "properties": {
                "from_account": {
                    "type": "string",
                    "description": "Source account number"
                },
                "to_account": {
                    "type": "string",
                    "description": "Destination account number"
                },
                "amount": {
                    "type": "number",
                    "description": "Amount to transfer"
                },
                "description": {
                    "type": "string",
                    "description": "Optional description for the transfer"
                }
            },
            "required": ["from_account", "to_account", "amount"]
        }
    },
    {
        "name": "get_account_statement",
        "description": "Get account statement for a specific date range",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {
                    "type": "string",
                    "description": "The account number"
                },
                "start_date": {
                    "type": "string",
                    "description": "Start date in YYYY-MM-DD format"
                },
                "end_date": {
                    "type": "string",
                    "description": "End date in YYYY-MM-DD format"
                }
            },
            "required": ["account_number", "start_date", "end_date"]
        }
    }
]

print("✓ Function schemas defined")
print(f"\nAvailable functions: {[f['name'] for f in FUNCTION_SCHEMAS]}")

## Part 4: Function Calling in Action

In [ ]:
# Map function names to actual functions
AVAILABLE_FUNCTIONS = {
    "get_account_balance": get_account_balance,
    "get_transaction_history": get_transaction_history,
    "transfer_funds": transfer_funds,
    "get_account_statement": get_account_statement
}

def run_conversation(user_message: str) -> str:
    """
    Run a conversation with function calling support
    """
    messages = [
        {
            "role": "system",
            "content": """You are a helpful banking assistant. You can help customers with:
            - Checking account balances
            - Viewing transaction history
            - Transferring funds between accounts
            - Getting account statements
            
            Always confirm details before performing transfers.
            Be professional and clear in your responses."""
        },
        {"role": "user", "content": user_message}
    ]
    
    # First API call - ask GPT what to do
    response = client.client.chat.completions.create(
        model=client.gpt4_deployment,  # Function calling works best with GPT-4
        messages=messages,
        functions=FUNCTION_SCHEMAS,
        function_call="auto"
    )
    
    response_message = response.choices[0].message
    
    # Check if GPT wants to call a function
    if response_message.function_call:
        # Extract function name and arguments
        function_name = response_message.function_call.name
        function_args = json.loads(response_message.function_call.arguments)
        
        print(f"\n🔧 Function Call Detected:")
        print(f"   Function: {function_name}")
        print(f"   Arguments: {json.dumps(function_args, indent=2)}")
        
        # Execute the function
        function_to_call = AVAILABLE_FUNCTIONS[function_name]
        function_response = function_to_call(**function_args)
        
        print(f"\n📊 Function Result:")
        print(f"   {json.dumps(function_response, indent=2)}")
        
        # Add function response to messages
        messages.append(response_message)
        messages.append({
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response)
        })
        
        # Second API call - get final response
        second_response = client.client.chat.completions.create(
            model=client.gpt4_deployment,
            messages=messages
        )
        
        return second_response.choices[0].message.content
    
    else:
        # No function call needed
        return response_message.content

# Test function calling
print("\n" + "="*80)
print("TEST 1: Check Balance")
print("="*80)

user_query = "What's the balance in account ACC12345?"
print(f"\nUser: {user_query}")
response = run_conversation(user_query)
print(f"\nAssistant: {response}")

In [ ]:
# Test 2: Transaction history
print("\n" + "="*80)
print("TEST 2: Transaction History")
print("="*80)

user_query = "Show me the last 5 transactions for account ACC12345"
print(f"\nUser: {user_query}")
response = run_conversation(user_query)
print(f"\nAssistant: {response}")

In [ ]:
# Test 3: Transfer (requires confirmation)
print("\n" + "="*80)
print("TEST 3: Fund Transfer")
print("="*80)

user_query = "Transfer $500 from ACC12345 to ACC67890 for rent payment"
print(f"\nUser: {user_query}")
response = run_conversation(user_query)
print(f"\nAssistant: {response}")

## Part 5: Multi-Step Function Calling

Sometimes we need to call multiple functions in sequence.

In [ ]:
def run_multi_step_conversation(user_message: str, max_iterations: int = 5) -> str:
    """
    Handle conversations that may require multiple function calls
    """
    messages = [
        {
            "role": "system",
            "content": """You are a banking assistant. You can call multiple functions to help users.
            For example, you might need to check balance before transferring funds."""
        },
        {"role": "user", "content": user_message}
    ]
    
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        response = client.client.chat.completions.create(
            model=client.gpt4_deployment,
            messages=messages,
            functions=FUNCTION_SCHEMAS,
            function_call="auto"
        )
        
        response_message = response.choices[0].message
        
        if response_message.function_call:
            function_name = response_message.function_call.name
            function_args = json.loads(response_message.function_call.arguments)
            
            print(f"Calling: {function_name}({function_args})")
            
            function_to_call = AVAILABLE_FUNCTIONS[function_name]
            function_response = function_to_call(**function_args)
            
            messages.append(response_message)
            messages.append({
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response)
            })
        else:
            # No more function calls needed
            return response_message.content
    
    return "Maximum iterations reached."

# Test multi-step
print("\n" + "="*80)
print("TEST: Multi-Step Function Calling")
print("="*80)

complex_query = """I want to transfer $1000 from my checking account (ACC12345) to my savings (ACC67890). 
But first, can you check if I have enough balance?"""

print(f"\nUser: {complex_query}")
response = run_multi_step_conversation(complex_query)
print(f"\nFinal Response: {response}")

## Part 6: Error Handling and Validation

In [ ]:
def safe_function_call(function_name: str, function_args: Dict) -> Dict:
    """
    Safely execute a function with validation and error handling
    """
    try:
        # Validate function exists
        if function_name not in AVAILABLE_FUNCTIONS:
            return {"error": f"Unknown function: {function_name}"}
        
        # Additional validation for transfers
        if function_name == "transfer_funds":
            amount = function_args.get("amount", 0)
            
            # Check daily limit (example: $10,000)
            if amount > 10000:
                return {
                    "error": "Transfer amount exceeds daily limit of $10,000",
                    "requires_approval": True
                }
            
            # Check if accounts are the same
            if function_args.get("from_account") == function_args.get("to_account"):
                return {"error": "Cannot transfer to the same account"}
        
        # Execute function
        function_to_call = AVAILABLE_FUNCTIONS[function_name]
        result = function_to_call(**function_args)
        
        return result
    
    except Exception as e:
        return {"error": f"Function execution failed: {str(e)}"}

# Test error handling
print("Test: Error Handling")
print("\n1. Invalid account:")
print(json.dumps(safe_function_call("get_account_balance", {"account_number": "INVALID"}), indent=2))

print("\n2. Exceeds limit:")
print(json.dumps(safe_function_call("transfer_funds", {
    "from_account": "ACC12345",
    "to_account": "ACC67890",
    "amount": 15000
}), indent=2))

print("\n3. Same account transfer:")
print(json.dumps(safe_function_call("transfer_funds", {
    "from_account": "ACC12345",
    "to_account": "ACC12345",
    "amount": 100
}), indent=2))

## 🎯 Practical Exercise 1: Bill Payment System

Create a bill payment function and integrate it with the chatbot.

Requirements:
- Function: `pay_bill(account_number, biller_name, amount, due_date)`
- Validate account balance before payment
- Support scheduled payments
- Return confirmation with reference number

In [ ]:
# TODO: Implement bill payment system
# Your code here:


## 🎯 Practical Exercise 2: Investment Portfolio Manager

Create functions for managing investment portfolios.

Functions needed:
- `get_portfolio_summary(account_number)`
- `buy_investment(account_number, investment_type, amount)`
- `sell_investment(account_number, investment_id, shares)`
- `get_investment_performance(account_number, period)`

In [ ]:
# TODO: Implement investment portfolio manager
# Your code here:


## 🎯 Practical Exercise 3: Fraud Detection Integration

Add fraud detection to the transfer function.

Requirements:
- Check transaction patterns
- Flag suspicious amounts
- Verify recipient
- Require additional authentication for high-risk transfers
- Log all checks for audit

In [ ]:
# TODO: Implement fraud detection
# Your code here:


## Summary

In this lab, you learned:

- **Function Calling Basics**: How GPT can intelligently call functions
- **Function Schemas**: Defining functions in JSON Schema format
- **Parameter Extraction**: GPT extracts parameters from natural language
- **Multi-Step Calls**: Handling complex workflows with multiple functions
- **Error Handling**: Validating inputs and handling failures gracefully
- **Security**: Implementing limits, validation, and audit logging

### Best Practices:

1. **Always validate** function inputs before execution
2. **Implement limits** (daily transfer limits, transaction sizes)
3. **Require confirmation** for sensitive operations
4. **Log everything** for audit and compliance
5. **Handle errors gracefully** with clear user messages
6. **Use GPT-4** for function calling (better than GPT-3.5)
7. **Test edge cases** thoroughly

### Security Considerations:

- Never expose sensitive data in function responses
- Implement authentication and authorization
- Rate limit function calls
- Monitor for suspicious patterns
- Use secure connections (HTTPS)
- Encrypt sensitive data
- Implement multi-factor authentication for high-value transactions

### Next Lab:

Lab 09: Embeddings and Semantic Search - Learn how to build intelligent search systems for banking documents.

---

**Instructor:** Manuela Larrea | manuela.larrea@idataglobal.com